# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = False
save_masks = False
print_loss = True
output_tensors = False
EPOCHS = 5
iteration = "-12"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 50
#val_batch_size = 100
num_batch = 2000 #25000 #11112
val_batch = 2
imageSize = 416
SAVE_LOCATION = "./data/models/model_test"
LOAD_LOCATION = "./data/models/model_best"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    # time
    since = time.time()
    best_model = None
    best_loss = math.inf
    
    outputs_dir = "data/outputs/results/" + str(datetime.now().date())
    os.makedirs(outputs_dir, exist_ok=True)
    
    # logs
    batch_log = logly.logly(True, num_epochs, num_batch)
    epoch_log = logly.logly(True, 1, num_epochs)
    batch_log_name = str(datetime.now().date()) + iteration
    epoch_log_name = "epoch - "+ str(datetime.now().date()) + iteration
    
    # model and sigmoid function
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
            
        epoch_loss = 0
        
        #Training    
        model.train()
        phase = 'train'
        for batch in range(num_batch):
            if print_loss:
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            running_loss = 0.0
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)

            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                batch_log.write(log_dir, batch_log_name)
            
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
        
        epoch_loss /= num_batch
        epoch_log.log(phase, 0, epoch, epoch_loss)
            
        # Validation
        model.eval()
        phase = 'val'
        val_loss = 0
        for batch in range(val_batch):
            if print_loss:
                print('Val-Batch {}/{}'.format(batch + 1, val_batch), flush=True)
            
            #BATCH TUPLE
#           inputs, labels, paths = next(iter(train_dataloaders))
            inputs, labels, paths = next(iter(val_dataloaders))
            inputs = inputs.to(device)

            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1
            
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                # statistics
                val_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                batch_log.write(log_dir, batch_log_name)
                
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
            
            if save_masks and phase == 'val':
                outputs = sigmoid_function(outputs)
                j = 0
                for locations in paths:
                    img = Image.open(locations)
#                     num = locations.replace("./data/train/photos/", "").replace(".png", "")
#                     num = locations.replace("./data/validation/photos/", "").replace(".png", "")
                    for threshold in [0.05, .25, .5, .75]:
                        img = data_factory.showMaskOnImage(img,outputs[j], threshold)
                        img.save(outputs_dir  + "/train-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
                    j += 1
                
        val_loss /= val_batch
        epoch_log.log(phase, 0, epoch, val_loss)
        
        epoch_log.write(log_dir, epoch_log_name)    

#             if output_tensors:
#                 return inputs, outputs, paths

        #save best copy of  model
        #if phase == 'val' and epoch_loss < best_loss:
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model, LOAD_LOCATION)
            torch.save(model, SAVE_LOCATION)
            torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
        
    #save current model
    torch.save(model,SAVE_LOCATION)
    batch_log.write(log_dir, batch_log_name)
    epoch_log.write(log_dir, epoch_log_name)
    
    
    time_elapsed = time.time() - since
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = UNet1(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    print('Single convolution model, no weights', flush=True)
#     model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
#     print('initializing model with random weights', flush=True)
#     torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)
#     torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.05, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  

loading weights from ./data/models/model_best


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/5
Batch 1/2000
loss: 0.6910278797149658
---------------
Batch 2/2000
loss: 0.6898556351661682
---------------
Batch 3/2000
loss: 0.6892218589782715
---------------
Batch 4/2000
loss: 0.6909759640693665
---------------
Batch 5/2000
loss: 0.689556896686554
---------------
Batch 6/2000
loss: 0.6906872987747192
---------------
Batch 7/2000
loss: 0.6871628165245056
---------------
Batch 8/2000
loss: 0.6918885111808777
---------------
Batch 9/2000
loss: 0.6907596588134766
---------------
Batch 10/2000
loss: 0.6876693964004517
---------------
Batch 11/2000
loss: 0.6897700428962708
---------------
Batch 12/2000
loss: 0.6892202496528625
---------------
Batch 13/2000
loss: 0.690406322479248
---------------
Batch 14/2000
loss: 0.6889196634292603
---------------
Batch 15/2000
loss: 0.6898349523544312
---------------
Batch 16/2000
loss: 0.6915280222892761
---------------
Batch 17/2000
loss: 0.6907559633255005
---------------
Batch 18/2000
loss: 0.6907112002372742
---------------
Batch 19/20

---------------
Batch 150/2000
loss: 0.6915447115898132
---------------
Batch 151/2000
loss: 0.6908801198005676
---------------
Batch 152/2000
loss: 0.6901577115058899
---------------
Batch 153/2000
loss: 0.689815878868103
---------------
Batch 154/2000
loss: 0.6916124820709229
---------------
Batch 155/2000
loss: 0.6924619674682617
---------------
Batch 156/2000
loss: 0.689303457736969
---------------
Batch 157/2000
loss: 0.6917300820350647
---------------
Batch 158/2000
loss: 0.6912032961845398
---------------
Batch 159/2000
loss: 0.6904258728027344
---------------
Batch 160/2000
loss: 0.6887694597244263
---------------
Batch 161/2000
loss: 0.6933677196502686
---------------
Batch 162/2000
loss: 0.6902794241905212
---------------
Batch 163/2000
loss: 0.6896818280220032
---------------
Batch 164/2000
loss: 0.6897681951522827
---------------
Batch 165/2000
loss: 0.6903427839279175
---------------
Batch 166/2000
loss: 0.6915243864059448
---------------
Batch 167/2000
loss: 0.68778800964

---------------
Batch 297/2000
loss: 0.689939022064209
---------------
Batch 298/2000
loss: 0.692042350769043
---------------
Batch 299/2000
loss: 0.691893994808197
---------------
Batch 300/2000
loss: 0.6919237971305847
---------------
Batch 301/2000
loss: 0.6925325989723206
---------------
Batch 302/2000
loss: 0.6902322769165039
---------------
Batch 303/2000
loss: 0.6918657422065735
---------------
Batch 304/2000
loss: 0.6900872588157654
---------------
Batch 305/2000
loss: 0.6908965706825256
---------------
Batch 306/2000
loss: 0.6920465230941772
---------------
Batch 307/2000
loss: 0.6911982297897339
---------------
Batch 308/2000
loss: 0.6898552775382996
---------------
Batch 309/2000
loss: 0.69105064868927
---------------
Batch 310/2000
loss: 0.6894193887710571
---------------
Batch 311/2000
loss: 0.6903731822967529
---------------
Batch 312/2000
loss: 0.6930542588233948
---------------
Batch 313/2000
loss: 0.6911981105804443
---------------
Batch 314/2000
loss: 0.69130933284759

---------------
Batch 444/2000
loss: 0.6915208697319031
---------------
Batch 445/2000
loss: 0.6918696761131287
---------------
Batch 446/2000
loss: 0.6940845847129822
---------------
Batch 447/2000
loss: 0.6903316974639893
---------------
Batch 448/2000
loss: 0.692169725894928
---------------
Batch 449/2000
loss: 0.692729651927948
---------------
Batch 450/2000
loss: 0.6914194226264954
---------------
Batch 451/2000
loss: 0.6897493004798889
---------------
Batch 452/2000
loss: 0.6888844966888428
---------------
Batch 453/2000
loss: 0.6911838054656982
---------------
Batch 454/2000
loss: 0.690852165222168
---------------
Batch 455/2000
loss: 0.6916252374649048
---------------
Batch 456/2000
loss: 0.6903632283210754
---------------
Batch 457/2000
loss: 0.6893031001091003
---------------
Batch 458/2000
loss: 0.6924078464508057
---------------
Batch 459/2000
loss: 0.6912733316421509
---------------
Batch 460/2000
loss: 0.6921113133430481
---------------
Batch 461/2000
loss: 0.691472947597

loss: 0.6904001235961914
---------------
Batch 591/2000
loss: 0.6920262575149536
---------------
Batch 592/2000
loss: 0.688746452331543
---------------
Batch 593/2000
loss: 0.6905756592750549
---------------
Batch 594/2000
loss: 0.690919816493988
---------------
Batch 595/2000
loss: 0.6904605627059937
---------------
Batch 596/2000
loss: 0.6929212808609009
---------------
Batch 597/2000
loss: 0.6926945447921753
---------------
Batch 598/2000
loss: 0.6923002600669861
---------------
Batch 599/2000
loss: 0.6928345561027527
---------------
Batch 600/2000
loss: 0.6904163956642151
---------------
Batch 601/2000
loss: 0.6897878050804138
---------------
Batch 602/2000
loss: 0.6907146573066711
---------------
Batch 603/2000
loss: 0.6900737881660461
---------------
Batch 604/2000
loss: 0.6893790364265442
---------------
Batch 605/2000
loss: 0.6913211345672607
---------------
Batch 606/2000
loss: 0.6907694339752197
---------------
Batch 607/2000
loss: 0.6914074420928955
---------------
Batch 608

...

---------------
Batch 1322/2000
loss: 0.6910784840583801
---------------
Batch 1323/2000
loss: 0.6892564296722412
---------------
Batch 1324/2000
loss: 0.6915470361709595
---------------
Batch 1325/2000
loss: 0.6913227438926697
---------------
Batch 1326/2000
loss: 0.6900174021720886
---------------
Batch 1327/2000
loss: 0.6893321871757507
---------------
Batch 1328/2000
loss: 0.6925948858261108
---------------
Batch 1329/2000
loss: 0.6886875629425049
---------------
Batch 1330/2000
loss: 0.6884465217590332
---------------
Batch 1331/2000
loss: 0.6937921047210693
---------------
Batch 1332/2000
loss: 0.691640317440033
---------------
Batch 1333/2000
loss: 0.6895454525947571
---------------
Batch 1334/2000
loss: 0.6921839118003845
---------------
Batch 1335/2000
loss: 0.6884182095527649
---------------
Batch 1336/2000
loss: 0.6911926865577698
---------------
Batch 1337/2000
loss: 0.689939558506012
---------------
Batch 1338/2000
loss: 0.6904329657554626
---------------
Batch 1339/2000
l

Batch 1466/2000
loss: 0.6915832757949829
---------------
Batch 1467/2000
loss: 0.6882644295692444
---------------
Batch 1468/2000
loss: 0.690046489238739
---------------
Batch 1469/2000
loss: 0.691314697265625
---------------
Batch 1470/2000
loss: 0.6908724308013916
---------------
Batch 1471/2000
loss: 0.6921746730804443
---------------
Batch 1472/2000
loss: 0.6902090907096863
---------------
Batch 1473/2000
loss: 0.6894955039024353
---------------
Batch 1474/2000
loss: 0.6909798979759216
---------------
Batch 1475/2000
loss: 0.6916981339454651
---------------
Batch 1476/2000
loss: 0.6889141201972961
---------------
Batch 1477/2000
loss: 0.6916245818138123
---------------
Batch 1478/2000
loss: 0.6918041110038757
---------------
Batch 1479/2000
loss: 0.6906009912490845
---------------
Batch 1480/2000
loss: 0.6913018822669983
---------------
Batch 1481/2000
loss: 0.6917704343795776
---------------
Batch 1482/2000
loss: 0.6894773840904236
---------------
Batch 1483/2000
loss: 0.690759718

loss: 0.6928955912590027
---------------
Batch 1611/2000
loss: 0.6926645636558533
---------------
Batch 1612/2000
loss: 0.6904402375221252
---------------
Batch 1613/2000
loss: 0.6910433173179626
---------------
Batch 1614/2000
loss: 0.6926887035369873
---------------
Batch 1615/2000
loss: 0.693529486656189
---------------
Batch 1616/2000
loss: 0.6910600662231445
---------------
Batch 1617/2000
loss: 0.6899959444999695
---------------
Batch 1618/2000
loss: 0.6938578486442566
---------------
Batch 1619/2000
loss: 0.6932840347290039
---------------
Batch 1620/2000
loss: 0.6930902004241943
---------------
Batch 1621/2000
loss: 0.6926656365394592
---------------
Batch 1622/2000
loss: 0.6913716197013855
---------------
Batch 1623/2000
loss: 0.6913081407546997
---------------
Batch 1624/2000
loss: 0.6906810998916626
---------------
Batch 1625/2000
loss: 0.6910628080368042
---------------
Batch 1626/2000
loss: 0.690380871295929
---------------
Batch 1627/2000
loss: 0.6910886764526367
--------

---------------
Batch 1755/2000
loss: 0.6921544075012207
---------------
Batch 1756/2000
loss: 0.6903654336929321
---------------
Batch 1757/2000
loss: 0.6903951168060303
---------------
Batch 1758/2000
loss: 0.691378116607666
---------------
Batch 1759/2000
loss: 0.691737949848175
---------------
Batch 1760/2000
loss: 0.6889300346374512
---------------
Batch 1761/2000
loss: 0.690427839756012
---------------
Batch 1762/2000
loss: 0.6932997703552246
---------------
Batch 1763/2000
loss: 0.6914012432098389
---------------
Batch 1764/2000
loss: 0.6914297342300415
---------------
Batch 1765/2000
loss: 0.6899561285972595
---------------
Batch 1766/2000
loss: 0.6889309883117676
---------------
Batch 1767/2000
loss: 0.6888828873634338
---------------
Batch 1768/2000
loss: 0.6887944936752319
---------------
Batch 1769/2000
loss: 0.6885929703712463
---------------
Batch 1770/2000
loss: 0.691679835319519
---------------
Batch 1771/2000
loss: 0.6905777454376221
---------------
Batch 1772/2000
los

Batch 1899/2000
loss: 0.6919248104095459
---------------
Batch 1900/2000
loss: 0.6933480501174927
---------------
Batch 1901/2000
loss: 0.6908712387084961
---------------
Batch 1902/2000
loss: 0.6915419101715088
---------------
Batch 1903/2000
loss: 0.691423773765564
---------------
Batch 1904/2000
loss: 0.6900439262390137
---------------
Batch 1905/2000
loss: 0.6912920475006104
---------------
Batch 1906/2000
loss: 0.6914217472076416
---------------
Batch 1907/2000
loss: 0.688908576965332
---------------
Batch 1908/2000
loss: 0.692386269569397
---------------
Batch 1909/2000
loss: 0.6896426677703857
---------------
Batch 1910/2000
loss: 0.690696656703949
---------------
Batch 1911/2000
loss: 0.6918988823890686
---------------
Batch 1912/2000
loss: 0.6937435865402222
---------------
Batch 1913/2000
loss: 0.6928152441978455
---------------
Batch 1914/2000
loss: 0.6928421854972839
---------------
Batch 1915/2000
loss: 0.6914688944816589
---------------
Batch 1916/2000
loss: 0.69010317325

# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [8]:
#save state model as model
model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
model.load_state_dict(torch.load(LOAD_LOCATION+'_test-2019-04-23-3'))
torch.save(model, LOAD_LOCATION)